<a href="https://colab.research.google.com/github/gohigh19/EembeddedBoard/blob/master/Copy_of_deep_surpervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

train_new_model = False

# Data columns
categorical_cols = ["Tool Type", "Brand", "Condition", "Location"]
numerical_cols = ["Rental Duration (days)", "Popularity Score"]

# Preprocess data
def preprocess_data(data):

    # Encode categorical_cols
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

    # Scale numerical columns
    scaler = StandardScaler()
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

    # Split data into features and target
    X = data.drop("Rental Price", axis=1)
    y = data["Rental Price"]

    return X, y, label_encoders, scaler

# Function to train the model
def train_model(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2):
    # Object neural network model
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer with a single neuron for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split, verbose=0)

    return model

# Function to make predictions
def predict_rental_price(model, new_data, label_encoders, scaler):
    # Preprocess the new data
    for col in categorical_cols:
        new_data[col] = label_encoders[col].transform(new_data[col])
    new_data[numerical_cols] = scaler.transform(new_data[numerical_cols])

    # Predict the rental price for the new data point using the trained model
    new_pred = model.predict(new_data)
    return new_pred[0][0]

model_file_path = "/content/drive/My Drive/price_prediction_supervised_dl/trained_model.h5"
if not train_new_model and os.path.exists(model_file_path):
    trained_model = keras.models.load_model(model_file_path)
else:
    # Load and preprocess training data
    csv_file_path = '/content/drive/My Drive/price_prediction_supervised_dl/training_data.csv'
    data = pd.read_csv(csv_file_path)
    X, y, label_encoders, scaler = preprocess_data(data)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

    # Train the model
    trained_model = train_model(X_train, y_train)

    # Save the trained model
    trained_model.save(model_file_path)
# Predict on test set
y_pred = trained_model.predict(X_test)

# Create new data for prediction
new_data = pd.DataFrame({
    "Tool Type": ["Hammer"],
    "Brand": ["ABC Tools"],
    "Condition": ["New"],
    "Rental Duration (days)": [2],
    "Location": ["New York"],
    "Popularity Score": [8]
})

# Predict the rental price for the new data point
predicted_price = predict_rental_price(trained_model, new_data, label_encoders, scaler)
print(f"Predicted rental price: {predicted_price}")


1/1 [==============================] - 0s 20ms/step
Predicted rental price: 4.943324565887451
